# Contents
In this tutorial you will get a brief introduction to the sea-ice component of climate models. You will learn how to
- _analyse the sea ice concentration simulated by climate model_ 
- _deal with graphical issues linked to irregular grids_
- _compare the sea ice extent from various simulations against the satellite data_

# Sea ice in Climate model simulations from CMIP5
 The Model Intercomparison Projects (MIP) are standard sets of concerted experiments in which different modelling groups produce  simulations of the climate system that are compared with each others and assessed against available observational data sets. The Climate Model Intercomparison Project (CMIP) was promoted by the World Climate Research Program (WCRP) to support the Intergovernmental Panel on Climate Change (IPCC) during the assessment reports (AR) and provide numerical projections of future global climate under various scenarios. It is now in phase 6 (CMIP6). CMIP1 and 2 were voluntary and not much coordinated. CMIP3 was the firs organized and concerted phase. The successive phase was renamed CMIP5 to synchronize the number with the cycle of the IPCC assessment reports. 
They are composed of a minimum set of simulations (tiers): 
* A control simulation, in which the climate model are forced with pre-industrial conditions (a constant atmospheric CO2 and other greenhouse gases). These simulations are very long (> 500 years), to produce the baseline climate. They are essential to estimate climate variability for detection and attribution studies.
* A 1% atmospheric CO$_2$ increase up to 2x the pre-industrial concentration and then stabilizes. This is a standard benchmark for models to estimate the "climate sensitivity", which is measured as the change in the mean Earth temperature for a doubling of CO2.
* An "historical" simulation (ofetn referred to as transient), where observed trajectories of GHGs and other radiatively active forcings are prescribed in the atmosphere. Other drivers such as volcanic eruptions are included in the annual changes of the chemical concentrations.
* A set of future projections, which are narratives of what the future atmospheric concentration will be based on a combination of socio-economical conditions that have been translated into concentration of active radiative forcings (called Reference Concentration Pathways since CMIP5).

Additional experiments have been done to assess the role of the various forcings, so it is possible to find historical simulations in which only the GHG concentrations have been prescribed and the ozone stays at the preindustrial level. In others, the solar activity can be prescribed as the only source of (astronomical) natural variability. 

Several CMIP data can be obtained from the Earth System Federation Grid website, which is an international consortium of IT experts and climate scientists specifically created to access and process data at peta and exa scales. It is composed of different nodes in different parts of the world, and one example is the one at the Lawrence Livermore Lab in the US:
https://esgf-node.llnl.gov/projects/esgf-llnl/

Data are organized per projects, and within each project there are different experiments with many variables. Variable names are standardized according to the [Climate&Forecast (CF) convention](https://cfconventions.org/Data/cf-standard-names.html), which simplifies the bulk analysis. For certain experiments, there can also be ensemble simulations. An ensemble is one possible climate realization which is constrained by the same boundary conditions but evolved differently mostly because of different initial conditions (like a perturbed ocean or atmosphere state). 

There are millions of data available, and the majority of climate studies makes use of a large number of different simulations, to include as many information as possible to constrain the uncertainties linked to assessing future conditions. We will only analyse a few model outputs that have already been pre-processed. If you download raw data from ESGF you will have to run a few steps to get them ready for analysis. Different climate models (or Earth System Model, ESM, in case they include a carbon and other biogeochemical components) are identified by an institute name and by a model name. One institute can have different models, for instance with different horizontal resolutions or different components. 

There are several sea-ice variables produced by climate model. Sea-ice concentration data from 4 models have been provided, with climatological data for the historical period 1979-2005, which coincides with the satellite data we analysed previously. **The models are global, have different spatial resolutions, and often _irregular grids_.** 
* CMCC (Centro EuroMediterraneo per i Cambiamenti Climatici, Italy): CMCC-CM **[irregular, low-res]**
* NOAA-GFDL (National Oceanic and Atmospheric Administration - Geophysical Fluid Dynamics Lab, USA): GFDL-CM3 **[regular, mid-res]**
* CCCMA (Canadian Centre for Climate Modelling and Analysis, Canada) CanESM **[regular, coarse-res]**
* CSIRO (Commonwealth Scientific and Industrial Research Organisation, Australia) ACCESS1-0 **[irregular, mid-res]**

For each one there is an additional file that contains the area of the ocean grid cell. Monthly and daily climatology have been computed from the orginal daily data.
Data are available on the course website in NetCDF format. There is one directory for each collection of monthly, daily and grid area data.

In [ ]:
# import the required modules
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cmocean # using this through the "cmo" variable, so must be imported
import cartopy.crs as ccrs

In [ ]:
CMIP5DIR='data/CMIP5/'
MODEL='CMCC-CM'
fname = f'sic_mon_{MODEL}_1979-2005_clim.nc'
ds = xr.open_dataset(CMIP5DIR+'monthly/'+fname)
ds

In [ ]:
ds.sic

In [ ]:
ax=plt.axes(projection=ccrs.Robinson(central_longitude=0))
field = ds.sic.isel(time=8)
cmap=plt.cm.get_cmap('cmo.ice', 10)
im = field.plot(ax=ax,x='lon',y='lat',
                cmap=cmap,vmin=0.,vmax=100.,
                cbar_kwargs={'shrink':0.5},
                transform=ccrs.PlateCarree())
plt.show()

>What is going on here?

This happens because many ocean models have an irregular grid. This is the [ORCA2 grid used by the NEMO model](https://www.nemo-ocean.eu/doc/node108.html), which is a tripolar grid where the grid singularity at the North Pole has been solved by duplicating it and shifting it on land. The ORCA2 grid has a nominal resolution of 2 degrees with smaller cells at the equator and in the Mediterranean. The additional grid cells have been taken from the land, hence the complex irregularity that creates issues with cartopy. 

![Irregular grid](https://ars.els-cdn.com/content/image/1-s2.0-S146350030300074X-gr1.jpg)
<img src="https://www.ncl.ucar.edu/Applications/Images/orca_2_2_lg.png" alt="drawing" width="400"/>

## Workaround for irregular data

Some software can handle irregular grid: matlab, ncl and the python package [iris](https://scitools-iris.readthedocs.io/en/stable/). Irregular grids are usually an issue for visualization and do not affect your analysis. It is often advisable to carry out the analysis on the native grid.

Remapping or regredding is usually the option taken by users, especially when they need to compare many different models. This can be done using `cdo` and its [remapping capabilities](https://code.mpimet.mpg.de/projects/cdo/embedded/index.html#x1-6180002.12). The following command remaps your input data file into a regular 1x1 degree grid using a bilinear interpolation:

```sh
cdo remapbil,r360x180 input.nc output.nc
```
We will now do a quick'n dirty remapping in python for visualization purposes. You should only use these kind of maps for your own analysis and not in thesis, reports or other publications. Remapping is a complex subject and should not be taken lightly!

We'll start from selecting one record, extracting the numpy arrays (limited to the Southern Hemisphere) and plotting with the matplotlib commands:

In [ ]:
field = ds.sic.isel(time=8).where(ds.lat<-40.)
x=field.lon.data
y=field.lat.data
z = field.data

In [ ]:
cmap=plt.cm.get_cmap('cmo.ice', 10)
im=plt.pcolormesh(x,y,z,cmap=cmap,vmin=0.,vmax=100.)
cb = plt.colorbar(im, orientation='horizontal', shrink=0.75)
plt.show()

The next step is to regrid your data as if they were a set of disconnected triplets (x,y,value) that needs to be mapped onto a regular grid using a triangulation method function called `griddata`
![griddata](https://scipython.com/static/media/examples/E8/gridinterp.png)
>This figure shows how the different interpolation methods change the shape and values of the gridded map

In [ ]:
lons = x.flatten()
lats = y.flatten()
ice = z.flatten()
# define the regular grid
lons_1d = np.arange(0,361,0.5)
lats_1d = np.arange(-80,-40,0.5)
lons_2d, lats_2d = np.meshgrid(lons_1d, lats_1d)

In [ ]:
lats_1d

In [ ]:
from scipy.interpolate import griddata
ice_regrid = griddata((lons, lats), ice, (lons_2d, lats_2d), method='nearest')

In [ ]:
data_crs = ccrs.PlateCarree()
map_proj = ccrs.SouthPolarStereo()

ax=plt.axes(projection=map_proj)
cmap=plt.cm.get_cmap('cmo.ice', 10)
im = ax.pcolormesh(lons_2d,lats_2d,ice_regrid,cmap=cmap,vmin=0.,vmax=100.,transform=data_crs)
cb = plt.colorbar(im, orientation='horizontal', shrink=0.75)
ax.set_extent([-180, 180, -90, -52], ccrs.PlateCarree())
ax.gridlines(draw_labels=True)           
plt.show()

## Sea-Ice Extent from CMIP5 models

The SIE should be computed on the original (native) grid, to preserve the grid values. We will first load the Antarctic SIE that we computed in the previous tutorial, which we will use to visually compare with the simulated SIE seasonal cycle from each model.

In [ ]:
NSIDC_sie=np.loadtxt('NSIDC_0051-sh-SIE-1979-2005.txt')

We also need to load the are of the ocean cell, which is different for each grid and made available by every modelling group. They are found in the folder `CMIP5/area`

In [ ]:
aname = f'areacello_fx_{MODEL}_historical_r0i0p0.nc'
dsa = xr.open_dataset(CMIP5DIR+'area/'+aname)
dsa

In [ ]:
field = ds.sic.where(ds.lat<-40,drop=True) # extract data south of 40S
ice = field.data/100. # extract concentration values as a numpy array and convert to a fraction
ice[ice==0]=np.nan
ice[ice<0.15] = 0.; ice[ice>=0.15] = 1. # assign presence/absence
ice.shape

In [ ]:
area = dsa.areacello.where(dsa.lat<-40,drop=True).data # extract area data south of 40S
area = area/1.e6 # convert to km^2
area.shape

In [ ]:
sie = np.nansum(ice*area,axis=(1,2))
print(sie.shape)

In [ ]:
plt.plot(range(1,25),np.tile(sie/1.e6,2),'-o',label=MODEL)
plt.xticks(range(1,25))
plt.xlabel('Months')
plt.ylabel('SIE [Million km$^2$]')
plt.plot(range(1,25),np.tile(NSIDC_sie/1.e6,2),'-o',label='NSIDC')
plt.legend()
plt.show()

We can put these instructions in a loop to compare all the models with each other

In [ ]:
themodels = ['CMCC-CM','ACCESS1-0','CanESM2','GFDL-CM3']
CMIP5_sie = np.zeros([12,len(themodels)])
for m,MODEL in enumerate(themodels):
    fname = f'sic_mon_{MODEL}_1979-2005_clim.nc'
    ds = xr.open_dataset(CMIP5DIR+'monthly/'+fname)
    aname = f'areacello_fx_{MODEL}_historical_r0i0p0.nc'
    dsa = xr.open_dataset(CMIP5DIR+'area/'+aname)
    field = ds.sic.where(ds.lat<-40,drop=True) # extract data south of 40S
    ice = field.data/100. # extract concentration values as a numpy array and convert to a fraction
    ice[ice==0]=np.nan
    ice[ice<0.15] = 0.; ice[ice>=0.15] = 1. # assign presence/absence
    area = dsa.areacello.where(dsa.lat<-40,drop=True).data # extract area data south of 40S
    area = area/1.e6 # convert to km^2
    CMIP5_sie[:,m] = np.nansum(ice*area,axis=(1,2))

In [ ]:
plt.plot(range(1,13),CMIP5_sie/1.e6,'-o',label=themodels)
plt.xticks(range(1,13))
plt.xlabel('Months')
plt.ylabel('SIE [Million km$^2$]')
plt.plot(range(1,13),NSIDC_sie/1.e6,'-x',label='NSIDC')
plt.legend()
plt.show()

We see that the selected models are quite good at representing the SIE seasonal cycle. The asymmetric growth rate is better captured by the GFDL model, which is however the one showing the most poleward extent.